In [60]:
import talib
import pandas as pd
import backtrader as bt

In [61]:
df = pd.read_excel(r'gbp_usd_1h.xlsx')

In [62]:
ma_functions = {
    'SMA' : talib.SMA,
    'EMA' : talib.EMA,
    'WMA' : talib.WMA,
    'DEMA' : talib.DEMA,
    'TEMA' : talib.TEMA,
    'TRIMA' : talib.TRIMA,
    'KAMA' : talib.KAMA,
    # 'MAMA' : talib.MAMA,
    'T3' : talib.T3,
}

target_size = 2.4
stop_size = 1.2
ma_period = 24

In [63]:
for name in ma_functions:
    df[name] = ma_functions[name](real = df['close'],timeperiod = ma_period)
    

In [64]:
def fibo_atr_v1(df : pd.DataFrame, fibo_n : int = 10) -> pd.DataFrame:
    _df = df.copy(deep=True)
    
    def fib(n):
        if n <= 1:
            return n
        return fib(n-1) + fib(n-2)

    time_periods = [fib(i) for i in range(1, fibo_n+1)]
    atr_cols = {f'atr_{i}' : i for i in time_periods}

    for col in atr_cols:
        _df[col] = talib.ATR(
            _df['high'],
            _df['low'],
            _df['close'],
            timeperiod=atr_cols[col],
        )

    _df['fibo_atr'] = _df[[i for i in atr_cols]].mean(axis=1)
    
    _df = _df.drop(columns = atr_cols)
    
    return _df

df = fibo_atr_v1(df)

In [67]:
def ma_status_transformer(ma_col_name : str) :

    def transformer(df):
        df [f'{ma_col_name}_status'] = df.apply(lambda row: 1 if row['close'] > row[ma_col_name] else 0, axis=1)
        return df

    return transformer


In [68]:
transformers = [ma_status_transformer(i) for i in ma_functions]

for transfomer in transformers:
    df = transfomer(df)


In [69]:
df

,Unnamed: 0,datetime,open,high,low,close,timeframe,SMA,EMA,WMA,...,T3,fibo_atr,SMA_status,EMA_status,WMA_status,DEMA_status,TEMA_status,TRIMA_status,KAMA_status,T3_status
0,0,2019-12-31 21:00:00+00:00,1.32508,1.32537,1.32350,1.32387,h1,NaN,NaN,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0
1,1,2019-12-31 22:00:00+00:00,1.32384,1.32465,1.32309,1.32446,h1,NaN,NaN,NaN,...,NaN,0.001560,0,0,0,0,0,0,0,0
2,2,2020-01-02 00:00:00+00:00,1.32485,1.32568,1.32392,1.32470,h1,NaN,NaN,NaN,...,NaN,0.001710,0,0,0,0,0,0,0,0
3,3,2020-01-02 01:00:00+00:00,1.32467,1.32604,1.32465,1.32493,h1,NaN,NaN,NaN,...,NaN,0.001495,0,0,0,0,0,0,0,0
4,4,2020-01-02 02:00:00+00:00,1.32490,1.32615,1.32465,1.32557,h1,NaN,NaN,NaN,...,NaN,0.001520,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35586,35586,2025-09-30 14:00:00+00:00,1.34367,1.34426,1.34234,1.34357,h1,1.343351,1.343279,1.343487,...,1.343865,0.001482,1,1,1,0,0,1,1,0
35587,35587,2025-09-30 15:00:00+00:00,1.34358,1.34508,1.34353,1.34408,h1,1.343312,1.343343,1.343546,...,1.343934,0.001448,1,1,1,0,0,1,1,1
35588,35588,2025-09-30 16:00:00+00:00,1.34416,1.34490,1.34124,1.34416,h1,1.343330,1.343408,1.343613,...,1.344000,0.002000,1,1,1,0,0,1,1,1
35589,35589,2025-09-30 17:00:00+00:00,1.34408,1.34659,1.34336,1.34606,h1,1.343511,1.343621,1.343832,...,1.344066,0.002107,1,1,1,1,1,1,1,1


In [16]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr', 'ma')  # add new lines
    params = (
        ('atr', -1),
        ('ma', -1),
    )

In [27]:
class MAStrategy1(bt.Strategy):

    def __init__(self):
        self.trades = []
        self.orders = []
        self._open_trades = {}

        self.call_counter = 0

    def calculate_volume(
            self,
            entry_price: float,
            stop_price: float,
            balance: int = 10000,
            risk: int = 1,
    ):
        self.call_counter += 1
        risk_amount = balance * (risk / 100)
        per_unit_risk = abs(entry_price - stop_price)
        units = risk_amount / per_unit_risk
        return int(units)

    def next(self):
        cal_ma_status = lambda close, ma: 'upper' if close > ma else 'under'

        ma = cal_ma_status(self.data.close[0], self.data.kama[0])

        if ma == "upper":
            price = self.data.close[0]
            entry_price = price
            stop_loss = entry_price - self.data.atr[0] * 4.8
            take_profit = entry_price + self.data.atr[0] * 2.4
            size = self.calculate_volume(
                entry_price,
                stop_loss,
                risk=2
            )
            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size=size,
                tradeid=1
            )

        # if current_ma_status == "under":
        #     price = self.data.close[0]
        #     entry_price = price
        #     stop_loss = entry_price + self.data.atr[0] * 2.4
        #     take_profit = entry_price - self.data.atr[0] * 4.8
        #     size = self.calculate_volume(
        #             entry_price,
        #             stop_loss,
        #             risk=2
        #         )
        #     self.sell_bracket(
        #         price=entry_price,
        #         stopprice=stop_loss,
        #         limitprice=take_profit,
        #         size=size
        #     )

    def notify_trade(self, trade: bt.trade.Trade):
        self.trades.append(trade)

    def notify_order(self, order):
        self.orders.append(order)




In [28]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [29]:
data = CustomPandasData(dataname=df.set_index("datetime"))

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(MAStrategy1)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
cerebro.broker.setcash(10000)
cerebro.broker.set_coo(True)
cerebro.broker.setcommission(commission=0.0, leverage=500)
result = cerebro.run()

-5228.768274050385